In [21]:
import re

import pandas as pd
from together import Together
from tqdm import tqdm


def preprocess_text(text):
    # Replace underscores and dashes with spaces
    # text = text.replace('_', ' ').replace('-', ' ')
    # Insert a space before all caps (but not at the start of the string)
    # text = re.sub(r'(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])', ' ', text)
    # Remove many whitespaces
    text = re.sub(r'\s+', ' ', text)

    return text


augmented_df = pd.read_csv("./data/augmented_datasets.csv")
data = augmented_df['augmented_description'].tolist()

print(preprocess_text(data[0]))

In [4]:
client = Together(api_key='d4668a226ce70586e35ae91a3a38db6f36bb09c4f5ee54a079551abacf3c1ea8')

In [14]:
preprocess_text(data[213])

In [23]:

for i in tqdm(range(len(data))):
# for i in tqdm(range(4000, len(data)):
    # if i = 417, continue to next iteration
    # if i != 417 and i != 1522 and i != 1566 and i != 2138 and i != 2279:
    stream = client.chat.completions.create(model="meta-llama/Llama-3-70b-chat-hf",
                                            messages=[{"role": "user", "content": f"""{preprocess_text(data[i])}
    Rewrite the text above fixing formatting errors, such as dashes, underscores, etc. in words (instead make the words human readable). If the features are human unreadable and you can't fix that, skip the features that are unreadable. Remove author names and names of people, but keep dataset name and tags. Remove numbers. Stick to the same content that's already provided in the text, and just give the text, no additional text before or after. Be as thorough and verbose as possible in reproducing the original text. If the text is too short, try to expand it by making the sentences longer with the main themes. Also, extract a few (3-10) keywords from the text at the end. Don't say "Here is the rewritten text" or anything like that, don't explain yourself, show don't tell, just give the final result!!!"""}],
                                            stream=True, temperature=0.7,
                                            top_p=0.95, top_k=40, repetition_penalty=1.1, max_tokens=None)
    output_text = ''
    for chunk in stream:
        output_text += chunk.choices[0].delta.content or ""

    with open(f'./data/dataset_{i}.txt', 'w') as f:
        f.write(str(output_text))